<a href="https://colab.research.google.com/github/ManishSinghJadoun/MLFLOW-DVC/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pwd

'/content'

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!apt-get install unrar

In [ ]:
!unrar x /content/drive/MyDrive/dataset.rar /content/

In [ ]:
import os
os.getcwd()

In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.io import read_image
import yaml
from PIL import Image

# Path to the dataset folder
dataset_dir = "D:\\dataclass"

# Define transformations for the images (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((1500, 1500)),  # Resize images to 224x224 (or any size suitable for your model)
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for pre-trained models
])

# Custom dataset to handle YAML files and images
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)  # List class subfolders
        self.image_paths = []
        self.labels = []

        # Load images and metadata (e.g., YAML) from each class folder
        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                # Get all image files in the class folder
                for file_name in os.listdir(class_dir):
                    if file_name.endswith(('.jpg', '.png', '.jpeg')):  # Check image extensions
                        image_path = os.path.join(class_dir, file_name)
                        self.image_paths.append(image_path)
                        self.labels.append(label)

                        # Optional: Load YAML metadata if needed
                        yaml_file = os.path.join(class_dir, file_name.replace('.jpg', '.yaml'))
                        if os.path.exists(yaml_file):
                            with open(yaml_file, 'r') as file:
                                metadata = yaml.safe_load(file)
                            # Process metadata as needed

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)  # Open image
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)  # Apply transformations

        return image, label

# Create an instance of the dataset
dataset = CustomImageDataset(root_dir=dataset_dir, transform=transform)

